In [68]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
import tensorflow as tf
torch.__version__, tf.__version__

('1.5.0', '2.2.0')

## convert data

In [23]:
from pathlib import Path
import pickle

def load(fn):
    with open(fn, "rb") as f:
        data = pickle.load(f, encoding='latin1')
    return data

def load1(fn):
    with open(fn, "rb") as f:
        data = pickle.load(f)
    return data

def save(data, fn):
    with open(fn, "wb") as f:
        pickle.dump(data, f)

In [21]:
old = "../../../../../py2/T-LSTM-master/Split0/"
newp = "../../data/"

In [24]:
for each in Path(old).glob("*"):
    nf = Path(newp) / each.name
    data = load(each)
    print(each.stem, data[1].shape)
    save(data, nf)
    nd = load1(nf)
    print(nf.stem, nd[1].shape)

label_train (2078, 2)
label_train (2078, 2)
label_test (891, 2)
label_test (891, 2)
hidden_ind_train (2078, 4, 1)
hidden_ind_train (2078, 4, 1)
hidden_ind_test (891, 4, 1)
hidden_ind_test (891, 4, 1)
elapsed_test (891, 1, 4)
elapsed_test (891, 1, 4)
elapsed_train (2078, 1, 4)
elapsed_train (2078, 1, 4)
data_test (891, 4, 529)
data_test (891, 4, 529)
accum_elapsed_train (2078, 4, 4)
accum_elapsed_train (2078, 4, 4)
accum_elapsed_test (891, 4, 4)
accum_elapsed_test (891, 4, 4)
data_train (2078, 4, 529)
data_train (2078, 4, 529)


## Translation tf to pytorch

In [11]:
tf.constant(1, dtype=tf.float32), tf.constant(1, dtype=tf.float32).shape

(<tf.Tensor: shape=(), dtype=float32, numpy=1.0>, TensorShape([]))

In [10]:
torch.tensor(2, dtype=torch.float32), torch.tensor(2, dtype=torch.float32).shape

(tensor(2.), torch.Size([]))

In [15]:
tf.ones([1, 10], dtype=tf.float32), tf.ones([1, 10], dtype=tf.float32).shape

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 TensorShape([1, 10]))

In [16]:
torch.ones((1,10), dtype=torch.float32), torch.ones((1,10), dtype=torch.float32).shape

(tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), torch.Size([1, 10]))

In [46]:
a = tf.math.log(tf.constant([[0.2, 0.5, 1, 5]]) + tf.constant(1.0))
b = tf.math.divide(tf.constant(1.0), a)
c = tf.ones([1, 4], tf.float32)
b.shape, tf.ones([1, 4]).shape
tf.matmul(b, tf.ones([4, 1]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[9.951922]], dtype=float32)>

In [50]:
a = torch.log(torch.tensor([[0.2, 0.5, 1, 5]]) + torch.tensor(1, dtype=torch.float32))
b = torch.div(torch.tensor(1, dtype=torch.float32), a)
c = torch.ones((4,1), dtype=torch.float32)
b @ c, torch.matmul(b, c)

(tensor([[9.9519]]), tensor([[9.9519]]))

In [52]:
Parameter(torch.Tensor(2, 2))

Parameter containing:
tensor([[0.1823, 0.4055],
        [0.6931, 1.7918]], requires_grad=True)

In [6]:
a = torch.rand((2,3,1))
b = torch.rand((2,3,4))
print(a, b)

c = torch.cat((a, b), dim=2)
print(c)

for t in range(3):
    print("index: ", t)
    print(c[:, t, :])
#     print(c[:, t, 0])
#     print(c[:, t, 1:])

tensor([[[0.9572],
         [0.4080],
         [0.6568]],

        [[0.3190],
         [0.7185],
         [0.3163]]]) tensor([[[0.8835, 0.2457, 0.1546, 0.7032],
         [0.1824, 0.5793, 0.6159, 0.9758],
         [0.2774, 0.1703, 0.1724, 0.2960]],

        [[0.3209, 0.1021, 0.8421, 0.5318],
         [0.5956, 0.3149, 0.6690, 0.6734],
         [0.1413, 0.3545, 0.4481, 0.5095]]])
tensor([[[0.9572, 0.8835, 0.2457, 0.1546, 0.7032],
         [0.4080, 0.1824, 0.5793, 0.6159, 0.9758],
         [0.6568, 0.2774, 0.1703, 0.1724, 0.2960]],

        [[0.3190, 0.3209, 0.1021, 0.8421, 0.5318],
         [0.7185, 0.5956, 0.3149, 0.6690, 0.6734],
         [0.3163, 0.1413, 0.3545, 0.4481, 0.5095]]])
index:  0
tensor([[0.9572, 0.8835, 0.2457, 0.1546, 0.7032],
        [0.3190, 0.3209, 0.1021, 0.8421, 0.5318]])
index:  1
tensor([[0.4080, 0.1824, 0.5793, 0.6159, 0.9758],
        [0.7185, 0.5956, 0.3149, 0.6690, 0.6734]])
index:  2
tensor([[0.6568, 0.2774, 0.1703, 0.1724, 0.2960],
        [0.3163, 0.1413, 0.3

In [65]:
tf.multiply(a, b)

<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.3412673 , 0.6143132 , 0.30103528, 0.42347988],
        [0.06384967, 0.01151462, 0.07089078, 0.04680625],
        [0.03913313, 0.01505405, 0.03377762, 0.04127943]],

       [[0.34445944, 0.3457046 , 0.11317814, 0.22907218],
        [0.35396236, 0.98564976, 0.82023317, 0.9243021 ],
        [0.13246745, 0.17909516, 0.09095483, 0.05182858]]], dtype=float32)>

In [67]:
torch.mul(a, b)

tensor([[[0.3413, 0.6143, 0.3010, 0.4235],
         [0.0638, 0.0115, 0.0709, 0.0468],
         [0.0391, 0.0151, 0.0338, 0.0413]],

        [[0.3445, 0.3457, 0.1132, 0.2291],
         [0.3540, 0.9856, 0.8202, 0.9243],
         [0.1325, 0.1791, 0.0910, 0.0518]]])

In [73]:
print(b.shape, b)
bu = torch.unsqueeze(b, 0)
bu.shape, bu

torch.Size([2, 3, 4]) tensor([[[0.5134, 0.9242, 0.4529, 0.6371],
         [0.4035, 0.0728, 0.4480, 0.2958],
         [0.9435, 0.3630, 0.8144, 0.9952]],

        [[0.9117, 0.9150, 0.2996, 0.6063],
         [0.3567, 0.9934, 0.8267, 0.9316],
         [0.4794, 0.6481, 0.3292, 0.1876]]])


(torch.Size([1, 2, 3, 4]),
 tensor([[[[0.5134, 0.9242, 0.4529, 0.6371],
           [0.4035, 0.0728, 0.4480, 0.2958],
           [0.9435, 0.3630, 0.8144, 0.9952]],
 
          [[0.9117, 0.9150, 0.2996, 0.6063],
           [0.3567, 0.9934, 0.8267, 0.9316],
           [0.4794, 0.6481, 0.3292, 0.1876]]]]))

In [75]:
buc = torch.cat([bu, bu], 0)
buc.shape, buc

(torch.Size([2, 2, 3, 4]),
 tensor([[[[0.5134, 0.9242, 0.4529, 0.6371],
           [0.4035, 0.0728, 0.4480, 0.2958],
           [0.9435, 0.3630, 0.8144, 0.9952]],
 
          [[0.9117, 0.9150, 0.2996, 0.6063],
           [0.3567, 0.9934, 0.8267, 0.9316],
           [0.4794, 0.6481, 0.3292, 0.1876]]],
 
 
         [[[0.5134, 0.9242, 0.4529, 0.6371],
           [0.4035, 0.0728, 0.4480, 0.2958],
           [0.9435, 0.3630, 0.8144, 0.9952]],
 
          [[0.9117, 0.9150, 0.2996, 0.6063],
           [0.3567, 0.9934, 0.8267, 0.9316],
           [0.4794, 0.6481, 0.3292, 0.1876]]]]))

In [79]:
buc.transpose(0, 1).contiguous()

tensor([[[[0.5134, 0.9242, 0.4529, 0.6371],
          [0.4035, 0.0728, 0.4480, 0.2958],
          [0.9435, 0.3630, 0.8144, 0.9952]],

         [[0.5134, 0.9242, 0.4529, 0.6371],
          [0.4035, 0.0728, 0.4480, 0.2958],
          [0.9435, 0.3630, 0.8144, 0.9952]]],


        [[[0.9117, 0.9150, 0.2996, 0.6063],
          [0.3567, 0.9934, 0.8267, 0.9316],
          [0.4794, 0.6481, 0.3292, 0.1876]],

         [[0.9117, 0.9150, 0.2996, 0.6063],
          [0.3567, 0.9934, 0.8267, 0.9316],
          [0.4794, 0.6481, 0.3292, 0.1876]]]])

In [64]:
bb = torch.rand(3,2,4)
tt =  torch.tensor([[[1],[1]], [[1],[1]],[[1],[1]]])

In [45]:
from enum import IntEnum
class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2
    
class NaiveLSTM(nn.Module):
    def __init__(self, input_sz: int, hidden_sz: int):
        super().__init__()
        self.input_size = input_sz
        self.hidden_size = hidden_sz
        # input gate
        self.W_ii = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_hi = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_i = Parameter(torch.Tensor(hidden_sz))
        # forget gate
        self.W_if = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_hf = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_f = Parameter(torch.Tensor(hidden_sz))
        # ???
        self.W_ig = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_hg = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_g = Parameter(torch.Tensor(hidden_sz))
        # output gate
        self.W_io = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_ho = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_o = Parameter(torch.Tensor(hidden_sz))
         
        self.init_weights()
     
    def init_weights(self):
        for p in self.parameters():
            if p.data.ndimension() >= 2:
                nn.init.xavier_uniform_(p.data)
            else:
                nn.init.zeros_(p.data)
         
    def forward(self, x: torch.Tensor, init_states=None):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size()
        hidden_seq = []
        if init_states is None:
            h_t, c_t = torch.zeros(self.hidden_size).to(x.device), torch.zeros(self.hidden_size).to(x.device)
        else:
            h_t, c_t = init_states
        for t in range(seq_sz): # iterate over the time steps
            x_t = x[:, t, :]
            i_t = torch.sigmoid(x_t @ self.W_ii + h_t @ self.W_hi + self.b_i)
            f_t = torch.sigmoid(x_t @ self.W_if + h_t @ self.W_hf + self.b_f)
            g_t = torch.tanh(x_t @ self.W_ig + h_t @ self.W_hg + self.b_g)
            o_t = torch.sigmoid(x_t @ self.W_io + h_t @ self.W_ho + self.b_o)
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            hidden_seq.append(h_t)
            
        hidden_seq = torch.cat(hidden_seq, dim=Dim.batch)
        print(hidden_seq.shape)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(Dim.batch, Dim.seq).contiguous()
        return hidden_seq

In [46]:
lstm = NaiveLSTM(4, 10)

In [48]:
label = torch.tensor([[0,1], [1,0], [0,1]])
lstm(bb), lstm(bb).shape

torch.Size([6, 10])
torch.Size([6, 10])


(tensor([[ 0.1361,  0.0406,  0.0749,  0.0387,  0.1307,  0.1851],
         [ 0.1331,  0.0736,  0.1242,  0.1694,  0.1198,  0.1751],
         [ 0.1248,  0.0456,  0.0743,  0.1025,  0.0458,  0.0917],
         [ 0.1198,  0.0404,  0.1143,  0.1304,  0.1379,  0.1660],
         [ 0.1378,  0.1078,  0.0768,  0.2108,  0.0996,  0.1583],
         [-0.0775, -0.0885, -0.0784, -0.1604, -0.1456, -0.1419],
         [-0.1670, -0.0864, -0.1816, -0.2897, -0.2577, -0.2607],
         [-0.1185,  0.0315, -0.1447, -0.0685, -0.1150, -0.1039],
         [ 0.1373, -0.0146,  0.1442,  0.1815,  0.1191,  0.1462],
         [ 0.1391,  0.0119,  0.1512,  0.1177,  0.1031,  0.0860]],
        grad_fn=<CopyBackwards>),
 torch.Size([10, 6]))

In [201]:
torch.argmax(lstm(b)[:, -1, :], 1)

torch.Size([3, 4])
torch.Size([3, 4])
torch.Size([1, 3, 4])
torch.Size([3, 4])
torch.Size([3, 4])
torch.Size([1, 3, 4])
torch.Size([4, 3, 4])


tensor([2, 1, 1], grad_fn=<NotImplemented>)

In [5]:
torch.tensor([[1,0], [0,1]],dtype=torch.float32).shape

torch.Size([2, 2])

In [192]:
torch.nn.BCEWithLogitsLoss()(torch.tensor([[0.1, 0.8],[0.3, 0.7]]), torch.tensor([[1,0], [0,1]],dtype=torch.float32))

tensor(0.7683)

In [6]:
class masked_softmax_cross_entropy_loss(nn.Module):
    def __init__(self, weight=None):
        super(masked_softmax_cross_entropy_loss, self).__init__()

    def forward(self, input, target):
        if not target.is_same_size(input):
            raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

        inputs = F.softmax(input, dim=1)
        print(inputs)
        print(target * torch.log(inputs))
        loss = -torch.sum(target * torch.log(inputs), 1)
        print(loss)
        loss = torch.unsqueeze(loss, 1)
        return loss
    
x = masked_softmax_cross_entropy_loss()(torch.tensor([[0.1, 0.8],[0.3, 0.7]]), torch.tensor([[1,0], [0,1]], dtype=torch.float32))
x, torch.mean(x)

tensor([[0.3318, 0.6682],
        [0.4013, 0.5987]])
tensor([[-1.1032, -0.0000],
        [-0.0000, -0.5130]])
tensor([1.1032, 0.5130])


(tensor([[1.1032],
         [0.5130]]),
 tensor(0.8081))

In [75]:
tf.nn.softmax_cross_entropy_with_logits(labels=[[1,0],[0,1]], logits=[[0.1, 0.8],[0.3, 0.7]])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.1031861, 0.5130153], dtype=float32)>

In [65]:
import torch
from torch.nn.parameter import Parameter
from torch import nn as N
from torch.nn import functional as F
import numpy as np


class TLSTMConfig:
    def __init__(self, input_dim, output_dim, hidden_dim, fc_dim, dropoutput_rate):
        self.dropout_prob = dropoutput_rate
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.fc_dim = fc_dim


class SoftmaxCrossEntropyLoss(N.Module):
    """
    equivalent implementation of tf.nn.softmax_cross_entropy_with_logits
    """

    def __init__(self, weight=None):
        super().__init__()

    def forward(self, inputs, targets):
        if not targets.is_same_size(inputs):
            raise ValueError("Target size ({}) must be the same as input size ({})".format(targets.size(),
                                                                                           inputs.size()))

        inputs = F.softmax(inputs, dim=1)
        loss = -torch.sum(targets * torch.log(inputs), 1)
        loss = torch.unsqueeze(loss, 1)
        return torch.mean(loss)


class TLSTMCell(N.Module):

    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.Wi = Parameter(torch.Tensor(input_dim, hidden_dim))
        self.Ui = Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.bi = Parameter(torch.Tensor(hidden_dim))

        self.Wf = Parameter(torch.Tensor(input_dim, hidden_dim))
        self.Uf = Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.bf = Parameter(torch.Tensor(hidden_dim))

        self.Wog = Parameter(torch.Tensor(input_dim, hidden_dim))
        self.Uog = Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.bog = Parameter(torch.Tensor(hidden_dim))

        self.Wc = Parameter(torch.Tensor(input_dim, hidden_dim))
        self.Uc = Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.bc = Parameter(torch.Tensor(hidden_dim))

        self.W_decomp = Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_decomp = Parameter(torch.Tensor(hidden_dim))

    def init_weights(self):
        for p in self.parameters():
            if p.data.ndimension() >= 2:
                N.init.xavier_uniform_(p.data)
            else:
                N.init.zeros_(p.data)

    def forward(self, x: torch.Tensor, time: torch.Tensor, prev_hidden_state=None):
        # x has three dim: batch size(pats), seq_size(encounters), v_dim(features)
        bz, seq_sz, v_dim = x.size()
        tbz, tseq_sz, tv_dim = time.size()
        assert bz == tbz and seq_sz == tseq_sz, \
            "feature and time seq have different batch size {}-{} or seq len {}-{}".format(bz, tbz, seq_sz, tseq_sz)

        # init hidden if no previous hidden
        if prev_hidden_state is None:
            h_t, c_t = torch.zeros(self.hidden_dim).to(x.device), torch.zeros(self.hidden_dim).to(x.device)
        else:
            h_t, c_t = prev_hidden_state

        # recurrent loop
        hidden_seq = []
        for i, t in enumerate(range(seq_sz)):
            x_t = x[:, t, :]  # (batch, input)
            # process time difference
            t_t = time[:, t, :]
            T = self.map_elapse_time(t_t)
            C_ST = torch.tanh(torch.matmul(c_t, self.W_decomp) + self.b_decomp)
            C_ST_dis = torch.mul(T, C_ST)
            c_t = c_t - C_ST + C_ST_dis
            # input gate
            i_t = torch.sigmoid(torch.matmul(x_t, self.Wi) + torch.matmul(h_t, self.Ui) + self.bi)
            # forget gate
            f_t = torch.sigmoid(torch.matmul(x_t, self.Wf) + torch.matmul(h_t, self.Uf) + self.bf)
            # output gate
            o_t = torch.sigmoid(torch.matmul(x_t, self.Wog) + torch.matmul(h_t, self.Uog) + self.bog)
            # cadidate MemCell
            C = torch.tanh(torch.matmul(x_t, self.Wc) + torch.matmul(h_t, self.Uc) + self.bc)
            # current MemCell
            c_t = f_t * c_t + i_t * C
            # current hidden state
            h_t = o_t * torch.tanh(c_t)
            # hidden_seq.append(torch.stack((c_h_t, C_t)))
            hidden_seq.append(h_t.unsqueeze(0))  # create extra dim for later concat (seq, batch, input)

        hidden_seq = torch.cat(hidden_seq, dim=0)  # concat to get the seq dim back (seq, batch, input)
        # hidden_seq = hidden_seq.transpose(0, 1).contiguous()  # (seq, batch, input) => (batch, seq, input)
        return hidden_seq, (h_t, c_t)
    
    def map_elapse_time(self, t):
        c1 = torch.tensor(1, dtype=torch.float32)
        c2 = torch.tensor(np.e, dtype=torch.float32)

        T = torch.div(c1, torch.log(t + c2))
        grid = torch.ones((1, self.hidden_dim), dtype=torch.float32)
        return torch.matmul(T, grid)

class TLSTM(N.Module):

    def __init__(self, config=None):
        super().__init__()
        # self.fc_dim = fc_dim
        # self.output_dim = output_dim
        self.tlstm = TLSTMCell(config.input_dim, config.hidden_dim)
        self.dropout_prob = config.dropout_prob
        self.fc_layer = N.Linear(config.hidden_dim, config.fc_dim)
        self.output_layer = N.Linear(config.fc_dim, config.output_dim)

        # we can use pytorch default implementation of binary loss function - BCEWithLogitsLoss
        # but we found the output did not exactly match the tf.nn.softmax_cross_entropy_with_logits with mean reduce
        # To make sure, results repeatable, we will use SoftmaxCrossEntropyLoss for now
        # self.loss_fct = N.BCEWithLogitsLoss()
        self.loss_fct = SoftmaxCrossEntropyLoss()

    def forward(self, feature, time, labels):
        # get raw logits
        seq, (h_t, c_t) = self.tlstm(feature, time)
        # seq = seq[-1, :, :]  # (seq, batch, input); get the last seq for classification
        last_state = h_t
        last_state = F.relu(self.fc_layer(last_state))
        last_state = F.dropout(last_state, p=self.dropout_prob)
        logits = self.output_layer(last_state)

        # measure loss
        loss = self.loss_fct(logits, labels)
        return loss, logits, torch.argmax(logits, dim=1)

In [73]:
config = TLSTMConfig(4, 2, 10, 5, 0.1)
t = TLSTM(config)
label = torch.tensor([[0,1], [1,0], [0,1]], dtype=torch.float32)
print(bb, tt)
t(bb, tt, label)

tensor([[[0.9261, 0.3319, 0.3081, 0.7040],
         [0.8074, 0.4562, 0.1841, 0.8379]],

        [[0.0504, 0.1540, 0.1602, 0.4930],
         [0.1777, 0.0024, 0.8727, 0.8469]],

        [[0.7725, 0.0687, 0.1997, 0.7045],
         [0.9790, 0.6107, 0.9465, 0.0355]]]) tensor([[[1],
         [1]],

        [[1],
         [1]],

        [[1],
         [1]]])


(tensor(0.7080, grad_fn=<MeanBackward0>),
 tensor([[-0.1574, -0.2433],
         [-0.1709, -0.2187],
         [-0.1709, -0.2187]], grad_fn=<AddmmBackward>),
 tensor([0, 0, 0], grad_fn=<NotImplemented>))

In [34]:
# list(t.named_parameters())
for n, p in t.named_parameters():
    print(n)

tlstm.Wi
tlstm.Ui
tlstm.bi
tlstm.Wf
tlstm.Uf
tlstm.bf
tlstm.Wog
tlstm.Uog
tlstm.bog
tlstm.Wc
tlstm.Uc
tlstm.bc
tlstm.W_decomp
tlstm.b_decomp
fc_layer.weight
fc_layer.bias
output_layer.weight
output_layer.bias


In [88]:
tbb = tf.transpose(bb, perm=[2,0,1])
ttbb = tf.transpose(tbb)
bb.shape, tbb.shape, ttbb.shape

(torch.Size([3, 2, 4]), TensorShape([4, 3, 2]), TensorShape([2, 3, 4]))

In [94]:
print(ttbb)

tf.Tensor(
[[[0.92605263 0.3318926  0.30814034 0.70401543]
  [0.05036551 0.1540097  0.16021532 0.49301505]
  [0.7725034  0.06866723 0.19968963 0.704536  ]]

 [[0.8074291  0.45615095 0.18412828 0.83786315]
  [0.17769796 0.00238603 0.8726631  0.8469192 ]
  [0.9789614  0.6106911  0.9465321  0.03552949]]], shape=(2, 3, 4), dtype=float32)


In [93]:
ptb = torch.transpose(bb, 1, 0)
ptb.shape, ptb

(torch.Size([2, 3, 4]),
 tensor([[[0.9261, 0.3319, 0.3081, 0.7040],
          [0.0504, 0.1540, 0.1602, 0.4930],
          [0.7725, 0.0687, 0.1997, 0.7045]],
 
         [[0.8074, 0.4562, 0.1841, 0.8379],
          [0.1777, 0.0024, 0.8727, 0.8469],
          [0.9790, 0.6107, 0.9465, 0.0355]]]))

## debuging tensors

In [38]:
# data
tr = load1("../../data/data_train.pkl")
ti = load1("../../data/elapsed_train.pkl")
l = load1("../../data/label_train.pkl")
len(tr), len(ti), len(l)

(10, 10, 10)

In [98]:
bb0 = bb[0]
print(bb0.shape)
trb = tf.reshape(bb0, [tf.shape(bb0)[0],tf.shape(bb0)[1],1])
trb, trb.shape

torch.Size([2, 4])


(<tf.Tensor: shape=(2, 4, 1), dtype=float32, numpy=
 array([[[0.92605263],
         [0.3318926 ],
         [0.30814034],
         [0.70401543]],
 
        [[0.8074291 ],
         [0.45615095],
         [0.18412828],
         [0.83786315]]], dtype=float32)>,
 TensorShape([2, 4, 1]))

In [101]:
bb0.view(bb0.shape[0], bb0.shape[1], 1)

tensor([[[0.9261],
         [0.3319],
         [0.3081],
         [0.7040]],

        [[0.8074],
         [0.4562],
         [0.1841],
         [0.8379]]])

In [103]:
for a, b, c in zip(tr, ti, l):
    print(a.shape, b.shape, c.shape)
    nb = np.reshape(b, [b.shape[0], b.shape[2]])
    print(nb.shape)
    nb = torch.tensor(nb)
    nb = torch.transpose(nb, 1, 0)
    print(nb.shape)
    nb = nb.view(nb.shape[0], nb.shape[1], 1)
    print(nb.shape)
    nb2 = np.reshape(b, [b.shape[0], b.shape[2], b.shape[1]])
    print(nb2.shape)
#     print(torch.tensor(a))
#     print(torch.tensor(b))
#     print(torch.tensor(c, dtype=torch.float32))
#     ta = torch.tensor(a)
    break

(536, 3, 529) (536, 1, 3) (536, 2)
(536, 3)
torch.Size([3, 536])
torch.Size([3, 536, 1])
(536, 3, 1)


In [5]:
b = torch.rand((2,3,2))
b, F.log_softmax(b, dim=2), torch.log(F.softmax(b, dim=2))

(tensor([[[0.2100, 0.1395],
          [0.0328, 0.4370],
          [0.0646, 0.3752]],
 
         [[0.1856, 0.2564],
          [0.2829, 0.5191],
          [0.1522, 0.5017]]]),
 tensor([[[-0.6585, -0.7290],
          [-0.9155, -0.5114],
          [-0.8605, -0.5499]],
 
         [[-0.7292, -0.6584],
          [-0.8182, -0.5820],
          [-0.8831, -0.5336]]]),
 tensor([[[-0.6585, -0.7290],
          [-0.9155, -0.5114],
          [-0.8605, -0.5499]],
 
         [[-0.7292, -0.6584],
          [-0.8182, -0.5820],
          [-0.8831, -0.5336]]]))

## cat+time series

In [111]:
# data format
d = (
    torch.tensor([1,0,0,1,1], dtype=torch.float32),
    torch.rand((1,182,10))
)
d, d[1].shape

((tensor([1., 0., 0., 1., 1.]),
  tensor([[[0.0289, 0.1848, 0.9356,  ..., 0.3355, 0.0893, 0.2524],
           [0.9931, 0.0061, 0.8767,  ..., 0.7025, 0.6984, 0.4814],
           [0.2147, 0.4837, 0.6908,  ..., 0.4290, 0.2049, 0.2470],
           ...,
           [0.1878, 0.6946, 0.4514,  ..., 0.6873, 0.1776, 0.1476],
           [0.6431, 0.9657, 0.5513,  ..., 0.9069, 0.1516, 0.5305],
           [0.3446, 0.0547, 0.4428,  ..., 0.4741, 0.8503, 0.1679]]])),
 torch.Size([1, 182, 10]))

In [108]:
# continuous vars
ecoder1 = nn.LSTM(10, 2, num_layers=1, batch_first=True)
t1 = ecoder1(d[1])[1][0]
print(t1)
t1 = torch.squeeze(t1, dim=0)

tensor([[[ 0.1406, -0.3870]]], grad_fn=<StackBackward>)


In [109]:
# categorical vars
ecoder2 = nn.Linear(5, 2)
t2 = ecoder2(d[0])
print(t2)
t2 = torch.unsqueeze(t2, dim=0)

tensor([-0.5153,  0.9061], grad_fn=<AddBackward0>)


In [110]:
torch.cat([t1, t2], dim=1) # combined representations

tensor([[ 0.1406, -0.3870, -0.5153,  0.9061]], grad_fn=<CatBackward>)